This project has comments to guide you through what's happening in the given code. Free free to comment your thoughts if you feel like.

# Importing necessary packages

In [ ]:
pip install Faker

# Creating Dataset having product Id and its cost respective to each hour of the day.

In [ ]:
import csv
from faker import Faker
import random

# Initialize the Faker generator
fake = Faker()

# Specify the number of products and hours
num_products = 10
num_hours = 24

# Create and open a CSV file for writing
with open('fake_price_dataset_with_min_cost.csv', 'w', newline='') as csvfile:
    fieldnames = ['product_id'] + [f'hour_{i}' for i in range(1, num_hours + 1)]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Generate fake records and write to the CSV file
    for product_id in range(1, num_products + 1):
        product_data = {
            'product_id': product_id,
        }

        base_price = round(random.uniform(50.0, 100.0), 2)  # Base price with lower variance
        hour_prices = []

        for hour in range(1, num_hours + 1):
            price_variation = random.uniform(-5.0, 5.0)  # Small random variation
            price = round(base_price + price_variation, 2)
            hour_prices.append(price)

        min_cost = round(sum(hour_prices) / num_hours, 2)  # Calculate average price as minimum cost

        # Adjust hourly prices to match the calculated minimum cost
        for hour in range(1, num_hours + 1):
            price_difference = min_cost - hour_prices[hour - 1]
            adjusted_price = round(min_cost + price_difference, 2)
            product_data[f'hour_{hour}'] = adjusted_price

        writer.writerow(product_data)

print(f'Generated fake price dataset with minimum cost for {num_products} products over {num_hours} hours in fake_price_dataset_with_min_cost.csv')


# Base price/Minimum cost of selling calculation

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('fake_price_dataset_with_min_cost.csv')

# Iterate through each product and calculate the average of hourly prices
for idx, row in data.iterrows():
    product_id = row['product_id']
    hourly_prices = row.iloc[1:].values  # Exclude the 'product_id' column
    average_price = sum(hourly_prices) / len(hourly_prices)

    # Calculate a base price as a percentage of the average price
    base_price = average_price * 0.95  # Adjust the factor as needed

    # Store the base price in a new column
    data.at[idx, 'base_price'] = base_price

# Save the updated DataFrame to a new CSV file
data.to_csv('product_price_with_base_price.csv', index=False)

print('Updated data saved to product_price_with_minSelling_price.csv')


# Creating a new database of product availibity.

In [ ]:
import pandas as pd
import random

In [ ]:
column = ['productId'] + [f'Hours_{i}' for i in range(1, 25)]
productId = [f'Product_{i}' for i in range (1, 11)]

In [ ]:
df = pd.DataFrame(columns = column)
df['productId'] = productId
for index, row in df.iterrows():
  for i in range (1, 25):
    df.at[index, f'Hours_{i}'] = random.randint(0, 1)

In [ ]:
df.to_csv('Dataset_Availibity.csv')

# Idea is to make the product more visible with lesser price when it is less available and its cost is relatively more than its usual/Previous cost.

In [ ]:
# Load the dataset
data = pd.read_csv('/content/product_price_with_base_price.csv')

# Create a new DataFrame to store suggestions
suggestions = data.copy()

# Iterate through the data and provide suggestions
for idx, row in data.iterrows():
    BasePrice = row['base_price']
    suggestions.at[idx, 'hour_1'] = "No previous data available"
    for hour in range(2, 25):  # Start from the second hour
        previous_hour_price = row[f'hour_{hour - 1}']
        current_hour_price = row[f'hour_{hour}']

        # Compare previous hour's price with current hour's price
        if current_hour_price > previous_hour_price and current_hour_price > BasePrice:
            suggestions.at[idx, f'hour_{hour}'] = f'Lower your price than {current_hour_price} and increase the visibility'
        else :
            suggestions.at[idx, f'hour_{hour}'] = f'Set the prices to between {BasePrice}-{current_hour_price}'

# Save the suggestions to a CSV file
suggestions.to_csv('price_change_suggestions.csv', index=False)

print('Suggestions saved to price_change_suggestions.csv')


# Model to get instructions based on availibility

In [ ]:
data = pd.read_csv('/content/Dataset_Availibity.csv')
InstructionToManagement = data.copy()
for index, row in data.iterrows():
  for i in range (1, 25):
    if data.at[index, f'Hours_{i}'] == 0:
      InstructionToManagement.at[index, f'Hours_{i}'] = 'Increase the visibility'
    else:
      InstructionToManagement.at[index, f'Hours_{i}'] = 'Do Nothing'
InstructionToManagement.to_csv('InstructionToManagement.csv', index=False)

## For future we can do prediction of availalibity and prediction of price of the product in rival websites so we can take actions according to that. We can add that to our problem statement.